inter_value between 0 and 1 for mixed voices

inter_value less than 0 or greater than 1 for crazy voices

In [54]:
#@title interpolate  rvc voice models
from collections import OrderedDict
import torch

def extract(ckpt):
            a = ckpt["model"]
            opt = OrderedDict()
            opt["weight"] = {}
            for key in a.keys():
                if "enc_q" in key:
                    continue
                opt["weight"][key] = a[key]
            return opt
ckpt_target="modelApth" #@param {type:"string"}
ckpt_target2="modelBpth" #@param {type:"string"}
inter_value = 0.5 #@param {type:"number"}
alpha1=inter_value
out_name="modelC" #@param {type:"string"}


ckpt1 = torch.load(ckpt_target2, map_location="cpu")
ckpt2 = torch.load(ckpt_target, map_location="cpu")
cfg = ckpt1["config"]

if "model" in ckpt1:
            ckpt1 = extract(ckpt1)
else:
            ckpt1 = ckpt1["weight"]
if "model" in ckpt2:
            ckpt2 = extract(ckpt2)
else:
            ckpt2 = ckpt2["weight"]

opt = OrderedDict()
opt["weight"] = {}
for key in ckpt1.keys():
            # try:
            if key == "emb_g.weight" and ckpt1[key].shape != ckpt2[key].shape:
                min_shape0 = min(ckpt1[key].shape[0], ckpt2[key].shape[0])
                opt["weight"][key] = (
                    alpha1 * (ckpt1[key][:min_shape0].float())
                    + (1 - alpha1) * (ckpt2[key][:min_shape0].float())
                ).half()
            else:
              opt["weight"][key] = (
                    alpha1 * (ckpt1[key].float()) + (1 - alpha1) * (ckpt2[key].float())
                ).half()
opt["config"] = cfg

opt["sr"] = "40k"
opt["f0"] = 1
opt["info"] = "100epoch"
torch.save(opt, out_name+".pth")

In [56]:
#@title Download interpolated  voices
from google.colab import files
files.download(out_name+".pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>